In [1]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import pickle as pkl
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score

In [2]:
nthu_train_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_X", 'rb'))
nthu_train_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_y", 'rb'))

nthu_test_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_X", 'rb'))
nthu_test_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_y", 'rb'))

In [3]:
nthu_train_y=np.array([int(nthu_train_y[i][0]/2) for i in range(len(nthu_train_y))])
nthu_test_y=np.array([int(nthu_test_y[i][0]/2) for i in range(len(nthu_test_y))])

In [4]:
#nthu_train_y = nthu_train_y.mean(axis=1)
#nthu_test_y = nthu_test_y.mean(axis=1)

In [5]:
#nthu_train_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_train_y, dtype = tf.float32))
#nthu_train_y_scaled = nthu_train_y_scaled.numpy()

In [6]:
#nthu_test_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_test_y, dtype = tf.float32))
#nthu_test_y_scaled = nthu_test_y_scaled.numpy()

In [7]:
idx =[]
np.random.seed(0)
for i in range(1000):
    idx.append(np.random.randint(420953))

In [8]:
n_steps_in, n_features, n_steps_out= 150, 8, 1

# define model
model = Sequential()
model.add(LSTM(100, activation='tanh', input_shape=(n_steps_in, n_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
# fit model
model.fit(nthu_train_X, np.ravel(nthu_train_y), batch_size=8, epochs=35, verbose=2)

Epoch 1/35
52620/52620 - 364s - loss: 0.4319
Epoch 2/35
52620/52620 - 364s - loss: 0.2091
Epoch 3/35
52620/52620 - 364s - loss: 0.1009
Epoch 4/35
52620/52620 - 363s - loss: 0.0593
Epoch 5/35
52620/52620 - 363s - loss: 0.0446
Epoch 6/35
52620/52620 - 364s - loss: 0.0360
Epoch 7/35
52620/52620 - 364s - loss: 0.0311
Epoch 8/35
52620/52620 - 364s - loss: 0.0283
Epoch 9/35
52620/52620 - 363s - loss: 0.0263
Epoch 10/35
52620/52620 - 366s - loss: 0.0251
Epoch 11/35
52620/52620 - 364s - loss: 0.0228
Epoch 12/35
52620/52620 - 363s - loss: 0.0227
Epoch 13/35
52620/52620 - 363s - loss: 0.0218
Epoch 14/35
52620/52620 - 363s - loss: 0.0274
Epoch 15/35
52620/52620 - 363s - loss: 0.0211
Epoch 16/35
52620/52620 - 363s - loss: 0.0236
Epoch 17/35
52620/52620 - 364s - loss: 0.0263
Epoch 18/35
52620/52620 - 363s - loss: 0.0897
Epoch 19/35
52620/52620 - 363s - loss: 0.2439
Epoch 20/35
52620/52620 - 363s - loss: 0.3421
Epoch 21/35
52620/52620 - 365s - loss: 0.3785
Epoch 22/35
52620/52620 - 363s - loss: 0.37

In [9]:
# demonstrate prediction
yhat = model.predict_classes(nthu_test_X, verbose=1)
print(yhat)

4101/4101 [==============================] - 14s 3ms/step
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [10]:
def logit(x):
    """ Computes the logit function, i.e. the logistic sigmoid inverse. """
    if (x<=0.5):
        return float(0)
    if (x>=0.880797):
        return float(2)
    return float(- tf.math.log(1. / x - 1.))

In [11]:
#yhat_inversed = np.array([logit(x) for x in yhat])

In [12]:
mean_squared_error(np.squeeze(yhat),nthu_test_y)

0.3692657054896821

In [13]:
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES\n")
    str_out += ("\n")

    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")

    #print AUC score
    auc = roc_auc_score(test, pred)
    str_out += ("AUC: {:.4f}\n".format(auc))
    str_out += ("\n")

    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")

    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

In [14]:
y_pred = np.squeeze(yhat)
results, false = display_test_scores(nthu_test_y, y_pred)
print(results)
y_pred.shape

TEST SCORES

ACCURACY: 0.6307

AUC: 0.6184

CONFUSION MATRIX:
[[23005 39353]
 [ 9105 59765]]

FALSE POSITIVES:
9105 out of 68870 positive labels (0.1322%)

FALSE NEGATIVES:
39353 out of 62358 negative labels (0.6311%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.72      0.37      0.49     62358
           1       0.60      0.87      0.71     68870

    accuracy                           0.63    131228
   macro avg       0.66      0.62      0.60    131228
weighted avg       0.66      0.63      0.60    131228



(131228,)

In [15]:
model.save("lstm_vanilla_v2.h5")